In [143]:
import requests
from bs4 import BeautifulSoup
import re
from unidecode import unidecode
import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.keys import Keys
import time

from selenium.common.exceptions import NoSuchElementException

from youtube_transcript_api import YouTubeTranscriptApi

import numpy as np

In [28]:
url = 'https://www.youtube.com/user/PowerfulJRE/videos'

chromedriver = '/Users/alex/Downloads/chromedriver 2'
driver = webdriver.Chrome(chromedriver)
driver.get(url)
try:
    myElem = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'body')))
    print("Page is ready!")
except TimeoutException:
    print("Loading took too much time!")

respData = driver.page_source
soup = BeautifulSoup(respData, 'html.parser')

Loading took too much time!


In [29]:
def scroll_down():
    body = driver.find_element_by_css_selector('body')
    body.send_keys(Keys.PAGE_DOWN)

In [30]:
links = []
transcripts = []


while len(links) <= 100:
    scroll_down()
    scroll_down()
                
    respData = driver.page_source
    soup = BeautifulSoup(respData, 'html.parser')
    links = soup.find_all('a', attrs={'id':'video-title'})
            
    if len(links) >= 100:
        for i, x in enumerate(links):
            yt_id = links[i]['href'].split('v=')[1]

            try:
                transcripts.append([links[i].text, YouTubeTranscriptApi.get_transcript(yt_id)])
                print(str(i) + '/100 transcripts completed')

            except:
                print('video ' + str(i) + ' does not have a transcript')
            

video 0 does not have a transcript
1/100 transcripts completed
2/100 transcripts completed
3/100 transcripts completed
4/100 transcripts completed
5/100 transcripts completed
6/100 transcripts completed
7/100 transcripts completed
8/100 transcripts completed
9/100 transcripts completed
10/100 transcripts completed
11/100 transcripts completed
video 12 does not have a transcript
13/100 transcripts completed
14/100 transcripts completed
15/100 transcripts completed
16/100 transcripts completed
17/100 transcripts completed
18/100 transcripts completed
19/100 transcripts completed
20/100 transcripts completed
21/100 transcripts completed
22/100 transcripts completed
23/100 transcripts completed
24/100 transcripts completed
25/100 transcripts completed
26/100 transcripts completed
27/100 transcripts completed
28/100 transcripts completed
29/100 transcripts completed
30/100 transcripts completed
31/100 transcripts completed
32/100 transcripts completed
33/100 transcripts completed
34/100 tra

In [104]:
video_captions = pd.DataFrame.from_dict(transcripts, orient='columns')

video_captions.rename(columns={1: 'raw_data'}, inplace=True)

In [120]:
video_captions['raw_data'][0][1]

{'text': "to see you guys it's always weird to",
 'start': 4.2,
 'duration': 5.22}

In [146]:
def pull_text(row):
    text = []
    
    for i in range(len(row)):
        text.append(row[i]['text'])
    final_text = ' '.join(text)
    return final_text



def pull_avg_duration(row):
    duration = []
    
    for i in range(len(row)):
        duration.append(row[i]['duration'])
    total_duration = np.array(duration).sum()
    return total_duration / len(row)

In [147]:
video_captions['text'] = video_captions['raw_data'].apply(pull_text)
video_captions['avg_duration_of_speaking'] = video_captions['raw_data'].apply(pull_avg_duration)

In [148]:
video_captions

,0,raw_data,text,avg_duration_of_speaking
0,Joe Rogan Experience #1485 - Krystal & Saagar,[{'text': 'here we go we're rolling what's up ...,here we go we're rolling what's up good to see...,4.411680
1,Joe Rogan Experience #1484 - Reggie Watts,[{'text': 'no better person to be here at the ...,no better person to be here at the end of the ...,4.544526
2,Joe Rogan Experience #1483 - Jesus Trejo,[{'text': 'jesus what's up man what's up broth...,jesus what's up man what's up brother good to ...,3.523731
3,Joe Rogan Experience #1482 - Jordan Jonas,[{'text': 'all right we're rolling thanks for ...,all right we're rolling thanks for doing this ...,4.938122
4,JRE MMA #96 with Justin Gaethje & Trevor Wittman,[{'text': 'boom trainer of the year Trevor Whi...,boom trainer of the year Trevor Whitman you mi...,4.297233
...,...,...,...,...
111,Joe Rogan Experience #1391- Tulsi Gabbard & Jo...,[{'text': 'all right here we go how did this g...,all right here we go how did this get started ...,4.543891
112,Joe Rogan Experience #1390 - Tim Dillon,"[{'text': 'three-two-one', 'start': 2.78, 'dur...",three-two-one ladies and gentlemen the preside...,4.170234
113,Joe Rogan Experience #1389 - Chris Kresser Deb...,[{'text': 'ladies and gentlemen Chris Kresser ...,ladies and gentlemen Chris Kresser Joe good EV...,4.702516
114,Joe Rogan Experience #1388 - Louie Psihoyos,[{'text': 'hmm all right here we go how are yo...,hmm all right here we go how are you Zebulon g...,4.543394


In [149]:
video_captions.to_csv('Latest_Joe_Rogan_Podcast_Captions.csv')

In [ ]:
class Multiple_YouTube_Scraper():
    def __init__(self, url):
        self.url = url
        
        chromedriver = '/Users/alex/Downloads/chromedriver 2'
        self.driver = webdriver.Chrome(chromedriver)
        
        self.driver.get(url)
        
        respData = self.driver.page_source
        self.soup = BeautifulSoup(respData, 'html.parser')
   
    def scroll_down(self):
        body = self.driver.find_element_by_css_selector('body')
        body.send_keys(Keys.PAGE_DOWN)
            
    def scrape_transcripts(self, number_of_transcripts):
        links = []
        while len(links) <= number_of_transcripts:
            self.scroll_down()
            self.scroll_down()

            respData = self.driver.page_source
            soup = BeautifulSoup(respData, 'html.parser')
            links = soup.find_all('a', attrs={'id':'video-title'})

            for i, x in enumerate(links):
                yt_id = links[i]['href'].split('v=')[1]

                try:
                    transcripts.append(YouTubeTranscriptApi.get_transcript(yt_id))
                    print(str(i) + '/100 transcripts completed')

                except:
                    pass